In [ ]:
using Plots
using LinearAlgebra
using Revise
using LatticeQM

# Tutorial: Haldane model

In [ ]:
lat = Geometries2D.honeycomb()
plot(lat, "sublattice"; supercell=Structure.Lattices.getneighborcells(lat, 70; halfspace=false, excludeorigin=false, innerpoints=true))

## Model

In [ ]:
# Graphene lattice
lat = Geometries2D.honeycomb()

# Haldane model
hops = Operators.graphene(lat; mode=:nospin, format=:dense)
Operators.addhaldane!(hops, lat, 0.05)

# Observables
valley = Operators.valley(lat; spinhalf=false)

nothing

### Bands

In [ ]:
ks = kpath(lat; num_points=200)
bands = getbands(hops, ks, [valley])
nothing

In [ ]:
Spectrum.getberry!(bands, hops, ks) # add berry curvature to band data
nothing

In [ ]:
plot(bands, 2; size=(320,200), colorbar=true)

In [ ]:
chern = sum(Spectrum.berry(Spectrum.wavefunctions(hops), 15, 15, [1])[2])

### Berry curvature map (Fukui method)

In [ ]:
berrycurvature = Spectrum.berry(Spectrum.wavefunctions(hops), 35, 35, [1]);

using Statistics

function BZheatmap(lat, cdata::T) where T<:AbstractMatrix{Float64}
    NX, NY = size(cdata) .+ 1

    p = plot()

    K(k::T) where T<:AbstractArray{Float64,N} where N = Structure.getB(lat) * k

    points = hcat([ K([(i+0.5)/NX, (j+0.5)/NY]) for i=0:NX-2 for j=0:NY-2]...)

    max = Statistics.quantile(abs.(cdata)[:], 0.98)
    clims = (-max,max)

    scatter(kpoints[1,:], kpoints[2,:]; 
    size=(200,400), markersize=2.0, colorbar=true, markercolor=:RdBu, 
    markerstrokewidth=0, clims=clims, marker_z=transpose(zcolor), 
    legend=:none, seriestype=:heatmap, aspectratio=:equal
    )
end

BZheatmap(lat, kgrid, berrycurvature)

### Phase diagram

In [ ]:
# Prepare parameters
PHI = LinRange(-π,π,200)
t2 = 0.2
M = 1.2 * t2 * LinRange(-3*2*sqrt(3), 3*2*sqrt(3), 200)

# Prepare output arrays
chern = zeros(size(PHI,1),size(M,1))

# Prepare system
lat = Geometries2D.honeycomb()
hops0 = Operators.graphene(lat; format=:dense)

for (i,phi) in enumerate(PHI), (j,Δ) in enumerate(M)
    hops = deepcopy(hops0)
    Operators.addsublatticeimbalance!(hops, lat, Δ)
    Operators.addhaldane!(hops, lat, t2; ϕ=phi)
    
    chern[i,j] = sum(Spectrum.berry(Spectrum.wavefunctions(hops), 10, 10, [1]))
end

In [ ]:
using LaTeXStrings
heatmap(PHI/π, M, round.(transpose(chern); digits=5); size=(400,180), color=:seismic, ylabel=L"$m/t_2$", xlabel=L"\phi/\pi", colorbar_title="Chern number")

### Haldane ribbon

In [ ]:
N=30
lat1 = Structure.reduceto1D(Geometries2D.honeycomb(), [[1, 1] [-N, N]])

plot(lat1, "sublattice"; supercell=[60], markersize=2, size=(700,250))

In [ ]:
# Hopping operator
hops1 = Operators.graphene(lat1; cellrange=1, format=:dense, mode=:nospin, tz=0.3, ℓinter=0.08, ℓintra=0.08)
Operators.addhaldane!(hops1, lat1, 0.3)

# Position operator along lattice vector a2
pos = Operators.positionalong(lat1, Structure.basis(lat1,2))

nothing

In [ ]:
ks = kpath(lat1; num_points=500)
bands1 = getbands(hops1, ks, pos)
p = plot(bands1, 1; ylabel="\$\\varepsilon/t\$", colorbar_title="transverse position", markersize=1.25, size=(500,300), cquantile=1.0, colorbar=true, csymmetric=false)